# MCP with Pydantic AI. Using MCP SSE Transport

In practice you probably won't ever need to implement the client yourself when we integrate tools into our agents: frameworks like Agents SDK and PydanticAI can do it.

Let's see how to do it with PydanticAI.

Make sure you have Pydantic AI with MCP support:

In [2]:
!uv add 'pydantic-ai[mcp]'

Resolved 259 packages in 1.22s                                       
Audited 239 packages in 27ms


At the moment of writing, there are problems with running this code on Windows. If you're on Windows, skip to "Running MCP with SSE".

We will run it in Terminal (because of async-io) - but the code with SSE we will create later will also work in Jupyter. If you only use Jupyter, also skip to "Running MCP with SSE".

Here's our script test.py - create it in a separate folder:

In [ ]:
# test.py
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStdio
from toyaikit.chat.interface import StdOutputInterface
from toyaikit.chat.runners import PydanticAIRunner

mcp_client = MCPServerStdio(
    command="uv",
    args=["run", "python", "main.py"],
    cwd="faq-mcp"
)


developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call. Use as many 
keywords from the user question as possible when making first requests.

Make multiple searches. Try to expand your search by using new keywords based on the results you
get from the search.

At the end, make a clarifying question based on what you presented and ask if there are 
other areas that the user wants to explore.
""".strip()


agent = Agent(
    name="faq_agent",
    instructions=developer_prompt,
    toolsets=[mcp_client],
    model='gpt-4o-mini'
)


chat_interface = StdOutputInterface()
runner = PydanticAIRunner(
    chat_interface=chat_interface,
    agent=agent
)


if __name__ == "__main__":
    import asyncio
    asyncio.run(runner.run())

We'll deal with dependencies using uv, so let's create an empty project:

In [ ]:
!uv init 
!uv add pydantic-ai[mcp] openai toyaikit

In [ ]:
!uv run python test.py

## Running MCP with SSE

Previously we used Standard Input/Output as the transport for MCP. We can also use HTTP (SSE) for that.

The only thing we need to change is how we run our server:

In [ ]:
mcp.run(transport="sse")

It's now available at "http://localhost:8000/sse".

When it comes to our code, we only need to change this part:

In [ ]:
from pydantic_ai.mcp import MCPServerSSE

mcp_client = MCPServerSSE(
    url='http://localhost:8000/sse'
)

How it will use HTTP for communication.

Now our Pydantic AI agents can use MCP!

In [4]:
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE
from toyaikit.chat.interface import StdOutputInterface
from toyaikit.chat.runners import PydanticAIRunner

mcp_client = MCPServerSSE(
    url='http://localhost:8000/sse'
)

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call. Use as many 
keywords from the user question as possible when making first requests.

Make multiple searches. Try to expand your search by using new keywords based on the results you
get from the search.

At the end, make a clarifying question based on what you presented and ask if there are 
other areas that the user wants to explore.
""".strip()


agent = Agent(
    name="faq_agent",
    instructions=developer_prompt,
    toolsets=[mcp_client],
    model='gpt-4o-mini'
)


chat_interface = StdOutputInterface()
runner = PydanticAIRunner(
    chat_interface=chat_interface,
    agent=agent
)

In [6]:
await runner.run();

You:  how do I install kafka?



--- Function Call ---
Function: search
Arguments: "{\"query\":\"install kafka\"}"
Result: [{'text': 'confluent-kafka: `pip install confluent-kafka` or `conda install conda-forge::python-confluent-kafka`\nfastavro: pip install fastavro\nAbhirup Ghosh\nCan install Faust Library for Module 6 Python Version due to dependency conflicts?\nThe Faust repository and library is no longer maintained - https://github.com/robinhood/faust\nIf you do not know Java, you now have the option to follow the Python Videos 6.13 & 6.14 here https://www.youtube.com/watch?v=BgAlVknDFlQ&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=80  and follow the RedPanda Python version here https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/06-streaming/python/redpanda_example - NOTE: I highly recommend watching the Java videos to understand the concept of streaming but you can skip the coding parts - all will become clear when you get to the Python videos and RedPanda files.', 'section': 'Module 6: stre

You:  stop


Chat ended.


Now we can use this MCP server with any MCP Client. For example, Cursor.

Add this server to .cursor/mcp.json:

```json
{
  "mcpServers": {
    "faqmcp": {
      "command": "uv",
      "args": [
        "run",
        "--project", "faq-mcp",
        "python",
        "faq-mcp/main.py"
      ]
    }
  }
}
```

If we run our MCP server with SSE transport, we configure it this way:

```json
{
  "mcpServers": {
    "faqmcp": {
      "url": "http://localhost:8000/sse"
    }
  }
}
```

If you don't get asked if you want to enable it, go to Preferenes -> Cursor settings -> MCP and Integrations, find your MCP server and enable it.

Examples of prompts:

- "Write code for module 1, check the FAQ for requirements"
- "Implement kafka connection with Python. Use FAQ to do comprehensive research first and then explain your choices."

Note: this isn't really a good usecase for Cursor. A more powerful usecase would be adding search for some frameworks. LLMs have some knowledge cutoff, while frameworks keep developing. So having access to fresh information is important.